# Researching and Writing blog posts with AI Agents 🤖

In [ ]:
!!pip install crewai PyPDF2

In [5]:
from dotenv import load_dotenv
from crewai import Agent, Task, Crew, Process
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from PyPDF2 import PdfReader

#### Put OPENAI_API_KEY in a `.env` file and load it 

In [6]:
# Load your OPENAI_API_KEY from your .env file
load_dotenv()

True

#### 🗯️Load the AI model (ChatGPT, LLama, Mistral, etc.)

In [26]:
### The model for the agents
model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.2)

### Or make it run local (with ollama)
# model = ChatOpenAI(
#     model="llama3:8b-instruct-q5_1",
#     base_url="http://localhost:11434/v1"
# )

#### Creating our own tool for reading PDF

In [29]:
# Tool for loading and reading PDF
@tool
def fetch_pdf_content(pdf_path: str):
    """
    Reads a PDF and returns the content
    """
    with open(pdf_path, 'rb') as f:
        pdf = PdfReader(f)
        text = '\n'.join(page.extract_text() for page in pdf.pages if page.extract_text())

    processed_text = re.sub(r'\s+', ' ', text).strip()
    return processed_text

#### We define three agents:
📄 Reads pdf and interprets it

✍️ Uses the content of PDF to write a blog post

🔥 Crafts a compelling title for the post

In [9]:
pdf_reader = Agent(
    role='PDF Content Extractor',
    goal='Extract and preprocess text from a PDF',
    backstory='Specializes in handling and interpreting PDF documents',
    verbose=True,
    tools=[fetch_pdf_content],
    allow_delegation=False,
    llm=model
)

article_writer = Agent(
    role='Article Creator',
    goal='Write a concise and engaging article',
    backstory='Expert in creating informative and engaging articles',
    verbose=True,
    allow_delegation=False,
    llm=model
)

title_creator = Agent(
    role='Title Generator',
    goal='Generate a compelling title for the article',
    backstory='Skilled in crafting engaging and relevant titles',
    verbose=True,
    allow_delegation=False,
    llm=model
)

#### We clearly define the task of each agent

In [30]:
def pdf_reading_task(pdf):
    return Task(
        description=f"Read and preprocess the text from the PDF at this URL: {pdf_url}",
        agent=pdf_reader,
        expected_output="Extracted and preprocessed text from a PDF",
    )

task_article_drafting = Task(
    description="Create a concise article with 8-10 paragraphs based on the extracted PDF content.",
    agent=article_writer,
    expected_output="8-10 paragraphs describing the key points of the PDF",
)

task_title_generation = Task(
    description="Generate an engaging and relevant title for the article.",
    agent=title_creator,
    expected_output="A Title of About 5-7 Words"
)

### We make a crew of these agents and kickoff 🚀

In [31]:
# USER INPUTS
pdf_url = r"2405.13817v1.pdf"

# Instantiate and run the crew
crew = Crew(
    agents=[pdf_reader, article_writer, title_creator],
    tasks=[pdf_reading_task(pdf_url), task_article_drafting, task_title_generation],
    verbose=2
)

# Execute the crew
result = crew.kickoff()

2024-05-24 22:48:33,589 - 139882310439552 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: PDF Content Extractor
 [INFO]: == Starting Task: Read and preprocess the text from the PDF at this URL: 2405.13817v1.pdf


> Entering new CrewAgentExecutor chain...
I need to use the fetch_pdf_content tool to extract and preprocess the text from the PDF at the given URL.

Action:
fetch_pdf_content

Action Input:
{"pdf_path": "2405.13817v1.pdf"}
 

Thermodynamic Natural Gradient Descent Kaelan Donatella∗,† Normal ComputingSamuel Duffield† Normal Computing Maxwell Aifer Normal ComputingDenis Melanson Normal ComputingGavin Crooks Normal ComputingPatrick J. Coles Normal Computing Abstract Second-order training methods have better convergence properties than gradient descent but are rarely used in practice for large-scale training due to their computa- tional overhead. This can be viewed as a hardware limitation (imposed by digital computers). Here we show that natural gradient descent (NGD), a second-order method, can have a similar computational complexity per 

### Display the Blog Post 🔥

In [32]:
final_article = f"\n\n{task_title_generation.output.exported_output}\n\n{task_article_drafting.output.exported_output}\n\n"
print("--------------------------")
print(final_article)

--------------------------


"Thermodynamic Natural Gradient Descent: Efficiency, Resilience, and Versatility in Optimization"

The Thermodynamic Natural Gradient Descent (TNGD) algorithm is a novel approach that utilizes the thermodynamic properties of an analog system to efficiently perform second-order optimization. In comparison to traditional first-order methods like Adam, TNGD has shown promising results in terms of convergence and efficiency. The algorithm's unique noise-resilience sets it apart and makes it a valuable tool for various tasks such as MNIST classification and language model fine-tuning for extractive question-answering.

One of the key advantages of TNGD is its ability to leverage the thermodynamic properties of an analog system, allowing for more efficient optimization compared to traditional methods. This approach has been shown to lead to faster convergence and improved performance in various optimization tasks. The experimental results presented in the text de